In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [2]:
%%writefile app.py
# ==================================================
# STREAMLIT APP — Employee Attrition Prediction
# ==================================================

import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# --------------------------------------------------
# LOAD DATA
# --------------------------------------------------

@st.cache_data
def load_data():
    df = pd.read_csv('Employee_Attrition.csv')
    return df

df = load_data()

st.title("🚀 Employee Attrition Prediction")
st.write("This app predicts whether an employee will leave the company or not using Logistic Regression, Random Forest, and Decision Tree.")

# --------------------------------------------------
# EDA Section (optional)
# --------------------------------------------------

with st.expander("📊 Explore Data"):
    st.write(df.head())

    st.subheader("Attrition Count")
    fig1, ax1 = plt.subplots()
    sns.countplot(x='Attrition', data=df, ax=ax1)
    st.pyplot(fig1)

    st.subheader("Correlation Heatmap")
    fig2, ax2 = plt.subplots(figsize=(10, 8))
    sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', ax=ax2)
    st.pyplot(fig2)

# --------------------------------------------------
# DATA PREPROCESSING
# --------------------------------------------------

df = df.drop(['EmployeeNumber', 'EmployeeCount', 'Over18', 'StandardHours'], axis=1, errors='ignore')

cat_cols = df.select_dtypes(include=['object']).columns

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

X = df.drop('Attrition', axis=1)
y = df['Attrition']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# --------------------------------------------------
# TRAIN MODELS
# --------------------------------------------------

lr = LogisticRegression()
lr.fit(X_train, y_train)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# --------------------------------------------------
# SIDEBAR USER INPUT
# --------------------------------------------------

st.sidebar.header('📌 Enter Employee Details')

def user_input_features():
    inputs = []
    for i, col in enumerate(X.columns):
        val = st.sidebar.number_input(f"{col}", min_value=0.0, step=1.0)
        inputs.append(val)
    data = np.array([inputs])
    return data

user_data = user_input_features()
user_data_scaled = scaler.transform(user_data)

# --------------------------------------------------
# PREDICT
# --------------------------------------------------

if st.sidebar.button("Predict"):
    lr_result = lr.predict(user_data_scaled)[0]
    rf_result = rf.predict(user_data_scaled)[0]
    dt_result = dt.predict(user_data_scaled)[0]

    st.subheader("✅ Prediction Results")
    st.write(f"**Logistic Regression:** {'Leaving' if lr_result == 1 else 'Staying'}")
    st.write(f"**Random Forest:** {'Leaving' if rf_result == 1 else 'Staying'}")
    st.write(f"**Decision Tree:** {'Leaving' if dt_result == 1 else 'Staying'}")

    st.success("Prediction Completed!")

# --------------------------------------------------
# END
# --------------------------------------------------


Writing app.py


In [3]:
! pip install streamlit pyngrok

In [4]:
!ngrok authtoken 2u8bzQfCvr5vlLyJvy0HCpQHUQ9_7H4DKQQBBGoPU8QbsaE7L

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
!streamlit run attrition_app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: attrition_app.py


In [6]:
import os
import time
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit is running at: {public_url}")

! streamlit run attrition_app.py --server.port 8501 --server.headless true &

time.sleep(3)

Streamlit is running at: NgrokTunnel: "https://0b82f13bcfb1.ngrok-free.app" -> "http://localhost:8501"
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: attrition_app.py
